# Machine Learning fo Reaserch

#### [Institute of Data Science at Maastricht University](https://www.maastrichtuniversity.nl/research/institute-data-science)
---

## Steps to choose a method based in a research question

- 1) Get to know your data  
- 2) Fit on train data  
- 3) Evaluate on test data  
- 4) Compare Methods  

![](../img/machinelearning.png)  
*This image is an extract from [SAP](https://blogs.sap.com/2016/05/11/go-for-the-intelligent-enterprise/) via [@JM_SAP](https://twitter.com/JM_SAP)*

---

In any case is easy to identify how to tackle a predictive probles, since is important to identify the method to respond the target variables.
    - Pattern: Anomaly Detection
    - Cicles: Time Series
    - Category: Classification
    - Quantity: Regression
    - Deep Features: Deep Learning

In [ ]:
import turicreate as tc
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

***`turicreate` [documentation](https://apple.github.io/turicreate/docs/api/index.html)***

### Load Airbnb Amsterdam Dataset

The Dataset is from [Tom Slee Blog](http://tomslee.net/airbnb-data-collection-get-the-data) and it shows the most recent listings (2017-07-22) from Amsterdam Airbnb Rooms, itself is extracted from [Inside Airbnb](http://insideairbnb.com/) where is possible to make a very nice visual analysis following the hypothesis: Airbnb claims to be part of the "sharing economy" and disrupting the hotel industry. However, data shows that the majority of Airbnb listings in most cities are entire homes, many of which are rented all year round - disrupting housing and communities.

*** Copyright Tom Slee, licensed under a [Creative Commons](https://creativecommons.org/licenses/by-nc/2.5/ca/deed.en_US) Attribution-NonCommercial 2.5 Canada License. ***

In [ ]:
df_rooms = pd.read_csv('https://s3.eu-west-3.amazonaws.com/pedrohserrano-datasets/airbnb_amsterdam.csv')

In [ ]:
df_rooms.head()

---
## Get to know your data

Exploratory data analysis (EDA) is part of any Data Science pipeline, is not Machine Learning itself thought is necessary to develop a method.  
EDA is the step where we start delving deeper into the data set beyond the outliers and errors. The knowledge of the dataset might be splitted in two scopes, and asking questions might help: 

**Intrinsic knowledge**
* How is my data distributed?
* Are there any correlations in my data?
* Are there any confounding factors that explain these correlations?
This is the stage where we plot all the data in as many ways as possible. Create many charts, but don't bother making them pretty — these charts are for internal use.

**Extrinsic knowledge**
* Make sure your data is encoded properly
* Make sure your data falls within the expected range, and use domain knowledge whenever possible to define that expected range
* Deal with missing data in one way or another: replace it if you can or drop it
* Never tidy your data manually because that is not easily reproducible

In [ ]:
df_rooms_cor = df_rooms[['room_type','reviews','overall_satisfaction','accommodates','bedrooms','price']]

In [ ]:
df_rooms_cor.describe()

In [ ]:
sb.pairplot(df_rooms_cor, hue='room_type')

---
## Fit on train data

Assured that our data is now as clean as we can make it — and armed with some cursory knowledge of the distributions and relationships in our data set — it's time to splitting the data into training and testing sets.

A **training set** is a random subset of the data that we use to train our models.

A **testing set** is a random subset of the data (mutually exclusive from the training set) that we use to validate our models on unforseen data.

Especially in sparse data sets like ours, it's easy for models to **overfit** the data: The model will learn the training set so well that it won't be able to handle most of the cases it's never seen before. This is why it's important for us to build the model with the training set, but score it with the testing set.

Note that once we split the data into a training and testing set, we should treat the testing set like it no longer exists: We cannot use any information from the testing set to build our model or else we're cheating.

Let's set up our data first.

#### Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [ ]:
sf_rooms = tc.SFrame('https://s3.eu-west-3.amazonaws.com/pedrohserrano-datasets/airbnb_amsterdam.csv')
train_data, test_data = sf_rooms.random_split(.8,seed=0)

### Target Feature 

In [ ]:
# Let's compute the mean of the Prices This is the Target value
prices = sf_rooms['price'] # extract the price column of the sales SFrame -- this is now an SArray
avg_price = prices.mean() # if you just want the average, the .mean() function
print ("Average price house: %.2f" % avg_price)

### Learning Multiple Models

In [ ]:
sf_rooms.column_names()

Now we will learn three (nested) models for predicting room prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: room_type, neighborhood, reviews
* Model 2: Add overall_satisfaction
* Model 3: Add accommodates, bedrooms
* Model 4: Just numerical, accommodates, bedrooms
* Model 5: Add reviews

In [ ]:
group_1_features = ['room_type', 'neighborhood', 'reviews']
group_2_features = group_1_features + ['overall_satisfaction']
group_3_features = group_2_features + ['accommodates', 'bedrooms']
group_4_features = ['accommodates', 'bedrooms']
group_5_features = group_4_features + ['reviews']

Now that you have the features, learn the weights for the three different models for predicting target = 'price'

In [ ]:
model_1 = tc.decision_tree_regression.create(train_data, target = 'price', features = group_1_features, validation_set = None)
model_2 = tc.random_forest_regression.create(train_data, target = 'price', features = group_2_features, validation_set = None)
model_3 = tc.random_forest_regression.create(train_data, target = 'price', features = group_3_features, validation_set = None)
model_4 = tc.linear_regression.create(train_data, target = 'price', features = group_4_features, validation_set = None)
model_5 = tc.linear_regression.create(train_data, target = 'price', features = group_5_features, validation_set = None)

In [ ]:
# To see the predictions
model_1.predict(train_data)

---
## Evaluate on test data

Now that you've learned five models and extracted the model metrics we want to evaluate which model is best.

In [ ]:
models = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
evaluation_train, evaluation_test = [], []
for model in models:
    rmse_train = tc.evaluation.rmse(train_data['price'], model.predict(train_data))
    rmse_test = tc.evaluation.rmse(test_data['price'], model.predict(test_data))
    evaluation_train.append(rmse_train)
    evaluation_test.append(rmse_test)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(models)), evaluation_test, color='r')
plt.plot(range(len(models)), evaluation_train, color='g')
plt.xticks(range(len(models)), ['Desicion Tree', 'RF + Satisfaction', 'RF All features', 'Linear Bedrooms', 'Linear + Reviews'])
plt.ylabel('RMSE')
plt.show()

- The closest between metrics of Train and Test the better
- The lowest the error the better performance of the model

In this case, we might pick up the model 2

Choose the crime you want to predict

---
### Iterate over the chosen model

In [ ]:
model_scores = []
for repetition in range(1000):
    train_data, test_data = sf_rooms.random_split(.8)
    model = tc.random_forest_regression.create(train_data, target = 'price', features = group_3_features, validation_set = None, verbose=False)
    rmse_test = tc.evaluation.rmse(test_data['price'], model.predict(test_data))
    model_scores.append(rmse_test)

In [ ]:
plt.figure(figsize=[12, 6])
plt.title('Evaluation Score Distribution')
sb.distplot(model_scores, color="g")

- Do we want to keep this model? 
- Shall we construct new features?
- What about more data exploration?
- Do we still having the same forst question?

---
### Documentation of the Method

The goal of construct a method is the reproducibility, that is why is important to document the procedures, taking into account versioning control of script for replicable science.

Notebooks like this one go a long way toward making our work reproducible. Since we documented every step as we moved along, we have a written record of what we did and why we did it — both in text and code.

Beyond recording what we did, we should also document what software and hardware we used to perform our analysis. This typically goes at the top of our notebooks so our readers know what tools to use.

[Sebastian Raschka](https://sebastianraschka.com/) created a handy notebook tool for this:

In [ ]:
#!pip install watermark
#only install once

In [ ]:
%load_ext watermark

In [ ]:
%watermark -a 'Maastricht University' -nmv --packages turicreate,pandas,seaborn,matplotlib